In [1]:
import tensorflow as tf

In [2]:
tf.config.list_physical_devices('GPU')

2023-05-03 17:21:12.864383: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudnn.so.8'; dlerror: libcudnn.so.8: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /packages/7x/jupyter/2022-04-15/lib


[]

2023-05-03 17:21:12.864422: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1850] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [3]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))


Num GPUs Available:  0


In [4]:
from tensorflow import keras

# Load MNIST dataset
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

# Reshape x-data
x_train = x_train.reshape(60000, 28, 28, 1)
x_test = x_test.reshape(10000, 28, 28, 1)

# Set aside raw test data for use with Akida runtime later
raw_x_test = x_test.astype('uint8')
raw_y_test = y_test

# Rescale x-data
a = 255
b = 0
input_scaling = (a, b)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train = (x_train - b) / a
x_test = (x_test - b) / a

11501568/11490434 [==============================] - 0s 0us/step


In [5]:
model_keras = keras.models.Sequential([
    keras.layers.Conv2D(
        filters=32, kernel_size=3, input_shape=(28, 28, 1), strides=2),
    keras.layers.BatchNormalization(),
    keras.layers.ReLU(),
    keras.layers.SeparableConv2D(
        filters=64, kernel_size=3, padding='same', strides=2),
    keras.layers.BatchNormalization(),
    keras.layers.ReLU(),
    keras.layers.Flatten(),
    keras.layers.Dense(10)
], 'mnistnet')

model_keras.summary()

Model: "mnistnet"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 13, 13, 32)        320       
                                                                 
 batch_normalization (BatchN  (None, 13, 13, 32)       128       
 ormalization)                                                   
                                                                 
 re_lu (ReLU)                (None, 13, 13, 32)        0         
                                                                 
 separable_conv2d (Separable  (None, 7, 7, 64)         2400      
 Conv2D)                                                         
                                                                 
 batch_normalization_1 (Batc  (None, 7, 7, 64)         256       
 hNormalization)                                                 
                                                          

2023-05-03 17:21:14.557177: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [6]:
from cnn2snn import check_model_compatibility

print("Model compatible for Akida conversion:",
      check_model_compatibility(model_keras))

Model compatible for Akida conversion: True


In [7]:
model_keras.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer='adam',
    metrics=['accuracy'])

model_keras.fit(x_train, y_train, epochs=10, validation_split=0.1)

score = model_keras.evaluate(x_test, y_test, verbose=0)
print('Test accuracy:', score[1])

Epoch 1/10
1688/1688 [==============================] - 8s 4ms/step - loss: 0.1665 - accuracy: 0.9491 - val_loss: 0.0822 - val_accuracy: 0.9765
Epoch 2/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.0678 - accuracy: 0.9794 - val_loss: 0.0750 - val_accuracy: 0.9787
Epoch 3/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.0496 - accuracy: 0.9843 - val_loss: 0.0550 - val_accuracy: 0.9870
Epoch 4/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.0407 - accuracy: 0.9869 - val_loss: 0.0587 - val_accuracy: 0.9865
Epoch 5/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.0326 - accuracy: 0.9889 - val_loss: 0.0578 - val_accuracy: 0.9848
Epoch 6/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.0268 - accuracy: 0.9915 - val_loss: 0.0645 - val_accuracy: 0.9845
Epoch 7/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.0246 - accuracy: 0.9916 - val_loss: 0.0965 - val_accuracy:

In [8]:
from cnn2snn import quantize

model_quantized = quantize(model_keras,
                           input_weight_quantization=8,
                           weight_quantization=4,
                           activ_quantization=4)
model_quantized.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (QuantizedConv2D)    (None, 13, 13, 32)        320       
                                                                 
 re_lu (QuantizedReLU)       (None, 13, 13, 32)        0         
                                                                 
 separable_conv2d (Quantized  (None, 7, 7, 64)         2400      
 SeparableConv2D)                                                
                                                                 
 re_lu_1 (QuantizedReLU)     (None, 7, 7, 64)          0         
                                                                 
 flatten (Flatten)           (None, 3136)              0         
                                                                 
 dense (QuantizedDense)      (None, 10)                31370     
                                                      

In [9]:
model_quantized.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer='adam',
    metrics=['accuracy'])

score = model_quantized.evaluate(x_test, y_test, verbose=0)
print('Test accuracy after 8-4-4 quantization:', score[1])

Test accuracy after 8-4-4 quantization: 0.7562999725341797


In [10]:
from cnn2snn import quantize_layer

model_quantized = quantize_layer(model_quantized, "re_lu_1", bitwidth=1)
model_quantized = quantize_layer(model_quantized, "dense", bitwidth=2)

model_quantized.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer='adam',
    metrics=['accuracy'])

score = model_quantized.evaluate(x_test, y_test, verbose=0)
print('Test accuracy after low bitwidth quantization:', score[1])

# To recover the original model accuracy, a quantization-aware training phase is required.

Test accuracy after low bitwidth quantization: 0.11509999632835388


In [11]:
model_quantized.fit(x_train, y_train, epochs=5, validation_split=0.1)

score = model_quantized.evaluate(x_test, y_test, verbose=0)
print('Test accuracy after fine tuning:', score[1])

Epoch 1/5
1688/1688 [==============================] - 9s 5ms/step - loss: 1.0809 - accuracy: 0.8693 - val_loss: 0.7541 - val_accuracy: 0.9652
Epoch 2/5
1688/1688 [==============================] - 8s 5ms/step - loss: 0.6903 - accuracy: 0.9638 - val_loss: 0.6856 - val_accuracy: 0.9728
Epoch 3/5
1688/1688 [==============================] - 8s 5ms/step - loss: 0.6366 - accuracy: 0.9676 - val_loss: 0.7033 - val_accuracy: 0.9710
Epoch 4/5
1688/1688 [==============================] - 8s 5ms/step - loss: 0.6434 - accuracy: 0.9685 - val_loss: 0.9035 - val_accuracy: 0.9705
Epoch 5/5
1688/1688 [==============================] - 8s 5ms/step - loss: 0.5936 - accuracy: 0.9699 - val_loss: 0.8747 - val_accuracy: 0.9698
Test accuracy after fine tuning: 0.9674999713897705


In [12]:
from cnn2snn import convert

model_akida = convert(model_quantized, input_scaling=input_scaling)
model_akida.summary()

accuracy = model_akida.evaluate(raw_x_test, raw_y_test)
print('Test accuracy after conversion:', accuracy)

# For non-regression purpose
assert accuracy > 0.96

                Model Summary                 
______________________________________________
Input shape  Output shape  Sequences  Layers
[28, 28, 1]  [1, 1, 10]    1          3     
______________________________________________

                  SW/conv2d-dense (Software)                  
______________________________________________________________
Layer (type)                  Output shape  Kernel shape    
conv2d (InputConv.)           [13, 13, 32]  (3, 3, 1, 32)   
______________________________________________________________
separable_conv2d (Sep.Conv.)  [7, 7, 64]    (3, 3, 32, 1)   
______________________________________________________________
                                            (1, 1, 32, 64)  
______________________________________________________________
dense (Fully.)                [1, 1, 10]    (1, 1, 3136, 10)
______________________________________________________________

Test accuracy after conversion: 0.9681000113487244
